# Referenzen und Adressen

<div class="prereq">
    <h3>Was man wissen sollte</h3>
    <div>
        Wir haben nochmal einen Blick auf <a class="prereq" href="/user-redirect/algoviz/lessons/05_Objekte/00_Objekte.ipynb">Klassen und Objekte</a> geworfen.
    </div>
</div>

Wir bereiten erstmal alles für unsere Experimente vor. Wir erzeugen ein Objekt der Klasse `SVG` und zwei Objekte der Klasse `Circle` in ihm. mit ihnen werden wir im Folgenden arbeiten.

In [14]:
#include<algoviz/SVG.hpp>
using namespace std;

AlgoViz::clear();

SVG zeichnung = SVG(400,400);

Circle kreis1 = Circle(20,20,10,&zeichnung);
Circle kreis2 = Circle(60,60,20,&zeichnung);

kreis2.setFill("red");

Jetzt haben wir zwei Objekte der Klasse `Circle`, die sich deutlich voneinander unterscheiden. Wir können beide auch unabhängig voneinander verändern und durch Methoden oder Operationen manipulieren.

Unser nöchstes Ziel ist es das `&` zu erklären, dass notwendig ist, wenn wir den Kreis einem SVG hinzufügen wollen.

Wir beginnen damit, dass wir eine Operation implementieren, die einen Kreis als Parameter erhält und ihn verändern soll. Und zwar soll sie seinen Radius verdoppeln und ihn blau machen. Das ist nicht weiter schwer.

In [2]:
void veraendere(Circle k) {
    k.setFill("blue");
    k.setRadius( 2* k.getRadius() );
    k.moveTo(200,200);
}

Wenn jetzt alles so funktioniert, wie man sich das denkt, müsste der folgende Aufruf den schwarzen Kreis blau färben und seinen Radius verdoppeln.

In [4]:
veraendere(kreis1);

Hm. Wie es scheint passiert nichts. Was haben wir falsch gemacht? Tatsächlich funktioniert das Ganze so, wie wir es bislang beschrieben haben. Um das zu sehen, bauen wir mal eine Wartephase ein. D.h. nachdem wir den Kreis verändert haben, warten wir auf einen Click.

In [10]:
void veraendere(Circle k) {
    k.setFill("blue");
    k.setRadius( 2* k.getRadius() );
    k.moveTo(200,200);
    
    zeichnung.waitForClick();
}

Führen Sie die nächste Zelle aus. Aber lesen Sie danach weiter, **bevor** Sie klicken!

In [11]:
veraendere(kreis1);

Wo kommt der dritte, blaue Kreis her? Klicken Sie jetzt.

...

Jetzt ist er wieder weg. Was ist passiert? Wir gehen mal die einzelnen Schritte durch.

**1. Schritt:** Mit der Anweisung `Circle kreis1 = Circle(20,20,10,&zeichnung)`haben wir ein Objekt der Klasse Kreis erzeugt, mit einem Ort und einem Radisu versehen und zu `zeichnung` hinzugefügt.

**2. Schritt:** `verandere(kreis1)` wird aufgerufen. Bei dem Aufruf wird der Parameter `k` als **lokale Variable** in `veraendere()` angelegt. Das Objekt `kreis1` wird in diese lokale Variable `k` **kopiert**. Dabei werden alle Attribute (Ort, Radisu, Farbe etc.) kopiert. D.h. es wird eine **identische** Kopie von `kreis1` gemacht. Diese Kopie wird in der lokalen Variable `k`  gespeichert. D.h. `kreis1` und `k` sind **Verschiedene** Instanzen mit denselben Attributen!

**3. Schritt:** `k` wird verändert. Die Kopie wird vergrößert, anders eingefärbt und verschoben. Dadurch erscheint der dritte Kreis, die ursprüngliche Kopie von `kreis1`.

**4. Schritt:** Wir warten auf den Klick. In dieser Zeit existiert das veränderte `k` und wird angezeigt.

**5. Schritt:** Nach dem Klick wird die Operation `veraendere()` beendet. Das führt dazu, dass die lokale Variable `k` aus dem Speicher gelöscht wird. Und das wiederrum führt dazu, dass der blaue Kreis wieder verschwindet!

In der Konsequenz hat `veraendere()` als nicht den übergebenen Kreis verändert, sondern eine Kopie, die beim Aufruf erzeugt und am Ende der Operation wieder **zerstört** wird. Warten wir nicht auf den Klick führt dies dazu, dass der ganze Prozess so schnell geht, dass wir ihn unter Umständen gar nicht sehen (oder wenn die Netzwerkverbindung langsam genug ist, der dritte Kreis nur kurz aufflackert).

Denselben Effekt können wir auch mit anderen Typen, z.B. `string` beobachten.

In [13]:
void change(string text) {
    text = "_" + text;
    cout << "in change: " << text << endl;
}

In [14]:
string meinText = "Hallo";
    
cout << "vor change: " << meinText << endl;
change(meinText);
cout << "nach change: " << meinText << endl;

vor change: Hallo
in change: _Hallo
nach change: Hallo


Wie man hier sieht, geht die Veränderung, die in `change()` vorgenommen wird wieder verloren. Beim Aufruf wird `meinText` Kopiert und die Kopie in der lokalen Variable `text` gespeichert. Diese wird verändert und am Ende der Operation wieder gelöscht. D.h. der Inhalt von `meinText` wurde gar nicht verändert.

Diesen Mechanismus nennt man **Call-By-Value**. Die Argumente beim Aufruf einer Operation werden kopiert und in der Operation mit den Kopien gearbeitet.

Damit stellt sich die Frage, wie wir Objekte in einer Operation verändern können? Und dazu verwendet man das Konzept der **Adressen**, **Referenzen** und **Zeiger**.

## Adressen - Wo Variablen wohnen

Wie wir bereits mehrmals festgestellt haben sind Variablen und auch Objekte reservierte Speicherbereiche in denen die jeweiligen Werte und Attribute gespeichert werden können. Um etwas besser zu verstehen, wie das geschieht, beschreiben wir das Vorgehen mit Hilfe von Paco, unserem Papier Computer.

Immer wenn Paco eine Operation aufruft, oder eine globale Variable speichern muss, erstellt er auf einem Blatt Papier einen Kasten (den reservierten Speicherbereich). Dieser Kasten wird mit einem Typ und einem Namen versehen. Allerdings macht er noch mehr. Jedes Blatt erhält eine eigene eindeutige Nummer (die blaue Zahl). Und jeder Kasten erhält ebenfalls eine eindeutige Nummer (die rote Zahl). Beide Zahlen haben eine feste Stellenzahl (in unserem Beispiel jeweils vier). Die **Adresse** der Variable ist dann die Konkatenation der beiden Zahlen. Als erstes die Seitennummer und als zweites die Nummer auf dem Blatt.

![Pacos Speicher](/user-redirect/algoviz/img/05_Objekte/PacoSpeicher.png)

Diese Adresse ermöglicht es jede Variable eindeutig zu identifizieren, auch wenn Sie auf einem anderen Blatt ist.

### Die Speicherorganisation 

Dieses Modell entspricht im Kern der Art, wie Betriebssysteme den Programmen Speicher zur Verfügung stehen. Der Speicher wird dabei als eine Sequenz von Bytes organisiert, die ähnlich wie die Einträge eines Arrays durchnummeriert sind. Außerdem ist er in der Regel auch in **Pages** oder **Speicherseiten** einer bestimmten Größe unterteilt.

Eine Variable oder ein Objekt ist dann ein **zusammenhängender Abschnitt** in einer dieser Seiten, ihre/seine Werte gespeichert werden können. Damit hat jede Variable und jedes Objekt eine **Adresse**, die Nummer der Speicherstelle, an dem sein reservierter Bereich beginnt. Und diese Adresse kann man sich anzeigen lassen. Dies macht der **Adressoperator `&`**. Man stellt ihn dem Namen der Variable voran und erhält so die Adresse. Mit `sizeof()`erhält man die Größe des reservierten Speicherbereichs.

In [4]:
#include <iostream>
using namespace std;  // nicht mehr gebraucht.

int variable;

// &variable ist die Adresse der Variable.
cout << "Adresse: " << &variable << endl;

// sizeof(variable) ergibt die Größe des Speicherbereichs.
cout << "Größe in Byte: " << sizeof(variable) << endl;

Adresse: 0x7ffb0c80d028
Größe in Byte: 4


Wie man sieht wird die Adresse wird als *Hexadezimalzahl* dargestellt.

Genauso kann man sich auch die Adresse und Größe eines beliebigen Objektes anzeigen lassen. Hier z.B. von einem `string`.

In [6]:
string text = "Dies ist ein Text";

// &variable ist die Adresse der Variable.
cout << "Adresse von text: " << &text << endl;

// sizeof(variable) ergibt die Größe des Speicherbereichs.
cout << "Größe von text in Byte: " << sizeof(text) << endl;

Adresse von text: 0x7ffb0c80d050
Größe von text in Byte: 32


Dabei fällt auf, dass die Größe des Strings nicht von seiner Länge abhängt. Stattdessen ist scheinbar jeder String 32 Byte groß. Das liegt daran, dass der eigentliche Inhalt **nicht** in dem für das Objekt reservierten Speicherbereich gespeichert wird sondern an anderer Stelle. Wie das funktioniert werden wir später noch sehen.

### Adressen von lokalen Variablen

Damit können wir uns auch die Adresse der lokalen Variablen in Operationen anzeigen lassen.

In [20]:
string meinText = "Hallo";

In [21]:
void operation(string lokalerText)  {
    cout << "Adresse von lokalerText: " << &lokalerText << endl;
}

In [22]:
cout << "Adresse von meinText : " << &meinText << endl;
operation(meinText);

Adresse von meinText : 0x7fd9caa47258
Adresse von lokalerText: 0x7ffc8edbc9c8


Wie man sieht haben die globale Variable `meinText` und `lokalerText` verschiedene Adressen, d.h. sie sind nicht identisch!

## Referenzparameter

Damit eine Operation auf das "richitge" Objekt zugreift, müsste es die Möglichkeit geben dafür zu sorgen, dass die lokalen Variabeln, bzw. die Parameter dieselbe **Adresse** verwenden. Und genau das tun die sogenannten
**Referenz-Parameter**.

Ein "normaler" wird zu einem solchen Referenz-Parameter, wenn man in der Signatur seinem Namen den Adressoperator `&` voranstellt. Dadurch wird beim Aufruf der Operation **nicht** der Wert kopiert, sondern der Parameter, also die lokale Variable dahinter, verwendet dieselbe **Adresse** wie das Argument des Aufrufs.

![Die Referenz](/user-redirect/algoviz/img/05_Objekte/PacoSpeicherReferenz.png)

Probieren wir das konkret aus.

In [22]:
#include <iostream>
using namespace std;

Die Operation `changeMitReferenz()`verwendet einen Referenz-Parameter, während `changeOhneReferenz()` einen "normalen" Parameter verwendet.

In [10]:
void changeMitReferenz(string &lokalerText) {
    lokalerText = "_" + lokalerText;    
    cout << "in changeMitReferenz: " << lokalerText << endl;
    cout << "Adresse von lokalerText : " << &lokalerText << endl;
}

In [11]:
void changeOhneReferenz(string lokalerText) {
    lokalerText = "_" + lokalerText;    
    cout << "in changeOneReferenz: " << lokalerText << endl;
    cout << "Adresse von lokalerText : " << &lokalerText << endl;
}

Probieren wir nun zuerst den Aufruf **ohne Referenz** aus. Dabei geschieht das, was wir bereits kennen. `lokalerText` ist eine Kopie mit einer anderen Adresse. Dadurch wird `globalerText` nicht verändert.

In [13]:
string globalerText = "Hallo";

cout << "globalerText : " << globalerText << endl;
cout << "Adresse von globalerText : " << &globalerText << endl;

changeOhneReferenz(globalerText);

cout << "globalerText : " << globalerText << endl;

globalerText : Hallo
Adresse von globalerText : 0x7ffb0c80d070
in changeOneReferenz: _Hallo
Adresse von lokalerText : 0x7ffe3ee7eab8
globalerText : Hallo


Und jetzt mit Referenz:

In [14]:
string globalerText = "Hallo";

cout << "globalerText : " << globalerText << endl;
cout << "Adresse von globalerText : " << &globalerText << endl;

changeMitReferenz(globalerText);

cout << "globalerText : " << globalerText << endl;

globalerText : Hallo
Adresse von globalerText : 0x7ffb0c80d090
in changeMitReferenz: _Hallo
Adresse von lokalerText : 0x7ffb0c80d090
globalerText : _Hallo


Wie man sieht hat die Variable `lokalerText` die gleiche Adresse, wie `globalerText`. Dadurch verändert die Operation tatsächlich die gleiche Variable und keine Kopie.

Der Adressoperator `&` vor dem Namen des Parameters einer Operation sorgt also dafür, dass statt einer Kopie dieselbe Adresse genutzt werden soll. Diesen Mechanismus nennt man **Call-By-Reference**.

### Referenz-Variablen

Neben Referenz-Parametern gibt es auch **Referenz-Variablen**. Diese Variablen speichern nicht, wie "normale" Variablen den Wert, sondern sie sind sozusagen ein Alias für eine andere Variable. Probieren wir das mal aus. Dazu deklarieren und initialisieren wir eine "normale" Variable.

In [ ]:
int variable = 42;

Im nächsten Schritt deklarieren wir eine Referenz-Variable, die wir auf `variable` verweisen lassen. Dabei müssen wir, genau wie bei einem Referenz-Parameter, den Adressoperator `&` vor den Namen stellen.

In [16]:
int &referenz = variable;

Wenn wir jetzt `variable` einen Wert zuweisen ändert sich auch der Wert von `referenz`.

In [19]:
cout << "variable = " << variable << endl;
cout << "referenz = " << referenz << endl;

variable = 84;

cout << "variable = " << variable << endl;
cout << "referenz = " << referenz << endl;

variable = 0
referenz = 0
variable = 84
referenz = 84


Und umgekehrt genau das Gleiche. Wenn wir `referenz` einen neuen Wert zuweisen, ändert sich auch `variable`.

In [20]:
cout << "variable = " << variable << endl;
cout << "referenz = " << referenz << endl;

referenz = 123;

cout << "variable = " << variable << endl;
cout << "referenz = " << referenz << endl;

variable = 84
referenz = 84
variable = 123
referenz = 123


Das Ganze liegt daran, dass die beiden Variablen **dieselbe** Adresse haben. Ändern wir den Wert, der an dieser Adresse gespeichert wird, indem wir über die eine Variable darauf zugreifen, so ändern wir natürlich auch den Wert, auf den wir über die andere Variable zugreifen.

In [23]:
cout << "&variable = " << &variable << endl;
cout << "&referenz = " << &referenz << endl;

&variable = 0x7ffb0c80d028
&referenz = 0x7ffb0c80d028


### Initialisierung von von Referenz-Variablen und -Parametern

Wenn wir eine Referenz-Variable deklarieren **muss** sie initilaisiert werden. Dabei können wir aber nur eine andere Variable gleichen Typs verwenden, keinen Ausdruck oder Konstanten Wert.

In [24]:
int variable = 42;
int &referenz = variable;   // Dies geht, da eine Variable zugewiesen wird

In [26]:
int &ausdruck = variable + 1;  // Dies geht nicht, da das Ergebnis der rechten Seite keine Variable ist.

input_line_37:2:7: error: non-const lvalue reference to type 'int' cannot bind to a temporary of type 'int'
 int &ausdruck = variable + 1;  // Dies geht nicht, da das Ergebnis der rechten Seite keine Variable ist.
      ^          ~~~~~~~~~~~~


Interpreter Error: 

In [27]:
int &konstante = 42;   // Dies geht auch nicht

input_line_38:2:7: error: non-const lvalue reference to type 'int' cannot bind to a temporary of type 'int'
 int &konstante = 42;   // Dies geht auch nicht
      ^           ~~


Interpreter Error: 

In [28]:
int &nichtinitialisiert;  // Geht auch nicht

input_line_39:2:7: error: declaration of reference variable 'nichtinitialisiert' requires an initializer
 int &nichtinitialisiert;  // Geht auch nicht
      ^~~~~~~~~~~~~~~~~~


Interpreter Error: 

Diese Einschränkungen gelten auch für Referenz-Parameter.

In [29]:
void mitReferenz(int &referenz) {
    referenz++;
}

In [30]:
mitReferenz(variable);    // Geht

In [31]:
mitReferenz(42);          // Geht nicht

input_line_42:2:2: error: no matching function for call to 'mitReferenz'
 mitReferenz(42);          
 ^~~~~~~~~~~
input_line_40:1:6: note: candidate function not viable: expects an l-value for 1st argument
void mitReferenz(int &referenz) {
     ^


Interpreter Error: 

In [32]:
mitReferenz(variable+1);  // Geht nicht

input_line_43:2:2: error: no matching function for call to 'mitReferenz'
 mitReferenz(variable+1);  // Geht nicht
 ^~~~~~~~~~~
input_line_40:1:6: note: candidate function not viable: expects an l-value for 1st argument
void mitReferenz(int &referenz) {
     ^


Interpreter Error: 

# Zusammenfassung

- Eine Variable/ein Objekt hat eine **Adresse**, die den Ort angbit, an dem es gepsichert wird.

- Die Adresse kann man sich mit dem Adressoperator `&` zurückgeben lassen.

- Beim Aufruf einer Operation werden in der Regel **Kopien** der Argumente gemacht und in den lokalen Variablen, die die Parameter darstellen, gespeichert. Diesen Vorgang nennt man **Call-By-Value** - "Aufruf mittels Wert".

- Wird dem Namen des Parameters in der Signatur der Adressoperator `&` vorangestellt wird der Wert des Arguments nicht kopiert, sondern derselbe Speicherbereich verwendet. Diesen Vorgang nennt man **Call-By-Reference** - "Aufruf mittels Referenz".

- Man kann auch **Referenz-Variablen** deklarieren, die auf andere Variablen verweisen.

# Die Nutzung von Referenz-Parametern

Mit Hilfe von Referenz-Parametern kann man z.B. Operationen implementieren, die mehrere Werte zurückgeben. Zum Beispiel kann man gleichzeitig **beide** Koordinaten eines Kreismittelpunktes zurückgeben.

In [1]:
#include <algoviz/SVG.hpp>
#include <iostream>
using namespace std;

AlgoViz::clear();

SVG zeichnung = SVG(400,400);
Circle kreis = Circle(70,30,5,&zeichnung);

In [2]:
void getPos(Circle k, int &x, int &y) {
    // x und y sind Referenz-Parameter
    // Die Koordinaten des Kreises werden in x und y gespeichert.
    x = k.getX();
    y = k.getY();
}

In [3]:
int xpos; 
int ypos;

kreis.moveTo(120,50);

getPos(kreis,xpos,ypos);

// Nach dem Aufruf enthalten die Variablen xpos
// und ypos Die Koordinaten des Kreismittelpunktes.
cout << xpos << "," << ypos << endl;

70,30


<div class="followup">
    <h3>Wo es weiter geht</h3>
    <div>Die Adressen eines Objektes oder eines Wertes können auch noch auf andere Art genutzt werden.
        Man kann sie als
        <a class="followup" href="/user-redirect/algoviz/lessons/05_Objekte/02_Zeiger.ipynb">Zeiger</a>
        auf Objekte und Daten verwenden.
    </div>
</div>   